In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
import datetime
def greeting():
    opts = { "hey" : ('Привет', 'Здравствуйте', 'Доброе утро', 'Добрый день', 'Добрый вечер', 'Доброй ночи')}

    now = datetime.datetime.now()
    #now += timedelta(hours = 3) # можешь проверить тут
    #print(now)
    if now.hour > 4 and now.hour <= 12 :
        greet = opts["hey"][2]
    if now.hour > 12 and now.hour <= 16 :
        greet = opts["hey"][3]
    if now.hour > 16 and now.hour <= 24 :
        greet = opts["hey"][4]
    if now.hour >= 0 and now.hour <= 4 :
        greet = opts["hey"][5]
    return greet

In [13]:
import itertools



# удаляет повторные буквы: пппрррииивеееет -----> привет

#def del_repeat(text):
#    return ''.join(c[0] for c in itertools.groupby(text))


# функция очищает текст
def clean(text):    
    text = text.lower() # приводит текст к нижнему регистру
    #text = del_repeat(text)
    cleaned_text = ''
    for ch in text:
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-—1234567890 ':
             cleaned_text = cleaned_text + ch
        
    return cleaned_text

# функция очищает текст
def clean1(text):    
    text = text.lower() # приводит текст к нижнему регистру
    #text = del_repeat(text)
    cleaned_text = ''
    for ch in text:
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя1234567890,.:;!?_-—() ':
             cleaned_text = cleaned_text + ch
    return cleaned_text

# транслитерация
def cyrillic2latin(text):
    cyrillic = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    latin = 'a|b|v|g|d|e|e|zh|z|i|i|k|l|m|n|o|p|r|s|t|u|f|kh|tc|ch|sh|shch||y||e|iu|ia'.split('|')
    trantab = {k:v for k,v in zip(cyrillic,latin)}
    newtext = ''
    for ch in text:
        casefunc =  str.capitalize if ch.isupper() else str.lower
        newtext += casefunc(trantab.get(ch.lower(),ch))
    return newtext

# №2
def cyrillic2latin2(text):
    cyrillic = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
    latin = 'a|b|v|g|d|e|e|zh|z|i|i|k|l|m|n|o|p|r|s|t|u|f|kh|tc|ch|sh|shch||y||e|iu|ia|A|B|V|G|D|E|E|Zh|Z|I|I|K|L|M|N|O|P|R|S|T|U|F|Kh|Tc|Ch|Sh|Shch||Y||E|Iu|Ia'.split('|')    
    return text.translate({ord(k):v for k,v in zip(cyrillic,latin)})


In [14]:
import pymorphy2
from nltk import word_tokenize

# функция очищает текст
def clean0(text):    
    text = text.lower() # приводит текст к нижнему регистру
    #text = del_repeat(text)
    cleaned_text = ''
    for ch in text:
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-—':
             cleaned_text = cleaned_text + ch
        else:
             cleaned_text = cleaned_text + ' '
    return cleaned_text

# очищаем запрос от стоп слов
def clear_text_stop_words(text, stop_words_list):
    text_tokens = word_tokenize(text)

    clear_data=[]
    for i in text_tokens:
        i = clean0(i)
        if i not in stop_words_list:
            clear_data.append(i)
    clear_data = sorted(set(clear_data))
    clear_text = ' '.join(clear_data)
    return clear_text    


# лемматизация 
morph = pymorphy2.MorphAnalyzer()

def text_to_normal_form(text, morph = morph):
    
    text2 = word_tokenize(text) # Список слов предложения s        
    s = ''
    for word in text2:
        word = morph.parse(word)[0].normal_form        
        s += (' ' + word)        
    
    s1 = ' '.join(sorted(list(set(s.split()))))    
    return s1.lstrip()



In [15]:
#собираю нормализованную БЗ в HIST_BOT_CONFIG_NORM.json
#продвинутая версия

from nltk.tokenize import sent_tokenize, word_tokenize


def file_intents_norm(file_in):
    #file_in = file_in
    file_content = ''
    
    file_content += '{\n'        
    file_content += ('    "intents": {\n')
    list_intents = list(HIST_BOT_CONFIG['intents'].keys())
    k = len(list_intents)
    for j in range(0,k): 
        intent = list_intents[j]
        file_content += ('       \"{}\" : {{\n'.format(intent))        
        file_content += ('           "examples" : [\n')

        m = len(HIST_BOT_CONFIG['intents'][intent]['examples'])
        for i in range(0,m):
            example = HIST_BOT_CONFIG['intents'][intent]['examples'][i]
            clear_example = clear_text_stop_words(example, stop_words)
            #clean_example = clean1(clear_example)            
            clean_norm_example = text_to_normal_form(clear_example)
            if i < m-1:
                file_content += ('              \"{}\",\n'.format(clean_norm_example))
            else:
                file_content += ('              \"{}\"\n'.format(clean_norm_example))
        
        file_content += ('         ],\n')
        file_content += ('           "responses" : [\n')

        #разбиваю response на список предложений  20.11
        list_responses = HIST_BOT_CONFIG['intents'][intent]['responses']
        n = len(list_responses)
        for p in range(0,n):
            response = list_responses[p]
            list_sentences = sent_tokenize(response, language="russian") #разбиваем справку на предложения
            m = len(list_sentences)
            for i in range(0,m):            
                sentence = list_sentences[i]            
                clear_sentence = clear_text_stop_words(sentence, stop_words)                 
                clean_norm_sentence = text_to_normal_form(clear_sentence)                
                
                if (p == n-1) and (i == m-1) :
                    file_content += ('              \"{}\"\n'.format(clean_norm_sentence))    
                else:
                    file_content += ('              \"{}\",\n'.format(clean_norm_sentence))

        file_content +=('         ]\n')
        if j < (k - 1):
            file_content += ('      },\n')
        else:
            file_content += ('      }\n')

    file_content += ('''



      },

    "default": [
      "Слишком много слов, напиши главное слово (словосочетание). Проверь, нет ли в слове опечатки?",
      "Что-то непонятно. Составь вопрос из двух-трех слов. Проверь, нет ли в слове опечатки?",
      "Сформулируйте попроще. Мне достаточно двух-трёх слов. Проверь, нет ли в слове опечатки?",      
      "Сформулируйте попроще или поточнее.  Мне будет достаточно одного-двух или трёх слов. Проверь, нет ли в слове опечатки?"
   ],
   "default1": [
      "Уточняющую информацию можно получить, если в запрос ввести непонятные слова из моих ответов"
   ]
}
        ''')
    
    

    #записываем конструкцию в файл
    with open(file_in, 'w', encoding='utf-8') as fin:
        fin.write(file_content)

In [16]:
# новые поисковые функции версия 2

import nltk
import random
from fuzzywuzzy import fuzz
import Levenshtein as lev
from functools import reduce
import math 




# --------------------движок--по--БаЗе-болтовни--------------------------------



def get_chat_intent(text):   
    for intent in BOT_CONFIG['intents'].keys():
        for example in BOT_CONFIG['intents'][intent]['examples']:
            clean_example = clean0(example)
            clean_text = clean0(text)                        
            if clean_example == clean_text:
                return intent
    return get_chat_intent1(text)



def get_chat_intent1(text):   
    for intent in BOT_CONFIG['intents'].keys():
        for example in BOT_CONFIG['intents'][intent]['examples']:
            clean_example = clean0(example)
            clean_text = clean0(text)
            clean_norm_example = text_to_normal_form(clean_example,morph)
            clean_norm_text = text_to_normal_form(clean_text,morph)
            if clean_norm_example == clean_norm_text:
                return intent            
            elif nltk.edit_distance(clean_example,clean_text)/len(clean_text) <.4:
                return intent
            elif nltk.edit_distance(clean_norm_example,clean_norm_text)/len(clean_norm_text)<.4:
                return intent
    return 'unknown_intent'

# ===========движок--по--БЗ-по-истории=================================================

# ---------------------функция сравнения строки и списка предложений------------------

#clean_norm_text - запрос, list_sentens = HIST_BOT_CONFIG_NORM['intents'][intent]['examples']
def metric_text_in_sentences(clean_norm_text,list_sentens):
    results_list = []
        
    metric = 0
    count_sentences = 0
    metric_max = 0
    count_max = 0
    for clean_norm_example in list_sentens: #HIST_BOT_CONFIG_NORM['intents'][intent]['examples']:            
            
            diff_len = abs(len(clean_norm_text) - len(clean_norm_example))
            
            list_clean_norm_example = clean_norm_example.split()
            list_clean_norm_text = clean_norm_text.split()
            
            len_example = len(list_clean_norm_example)
            len_text = len(list_clean_norm_text)
            
            # для каждого чистого экзампла считается 3 метрики Левенштейна
            dist_norm0 = fuzz.token_sort_ratio(clean_norm_example,clean_norm_text)
            dist_norm1 = lev.ratio(clean_norm_example,clean_norm_text)*100
            dist_norm = fuzz.token_set_ratio(clean_norm_text, clean_norm_example)
            if dist_norm > 60:
                print(clean_norm_text,'|', clean_norm_example,'|', dist_norm)
            total_quadr_aver_dist = ((dist_norm0**2+dist_norm1**2+dist_norm**2)/3)**(1/2)
            
            if (dist_norm0*dist_norm1*dist_norm) and (total_quadr_aver_dist > 70):
                
                total_harm_aver_dist = 3/(1/dist_norm0+1/dist_norm1+1/dist_norm)
                total_geom_aver_dist = (dist_norm0*dist_norm1*dist_norm)**(1/3)
                total_sum_aver_dist = (dist_norm0+dist_norm1+dist_norm)/3                
                
                
                total_diff_metric = total_quadr_aver_dist - total_harm_aver_dist           
                
                if (len_text <= len_example) and (dist_norm > 90):
                    metric_max += dist_norm0
                    count_max += 1
                    
                             
                elif (len_text <= len_example) and (dist_norm > 75):
                #if (dist_norm > 75):
                    # кумулятивная метрика экзамплов для одного интента
                    metric += dist_norm
                    count_sentences += 1
                results_list.append([dist_norm, clean_norm_example])
                    #results_exam.append([total_quadr_aver_dist, clean_norm_example])                    
                    
            elif (len_text == len_example == 1) and (dist_norm0 > 65):
                
                if dist_norm0 > 90:
                    metric_max += dist_norm0
                    count_max += 1
                else:
                #print('dist_norm =',dist_norm, 'dist_norm0 =', dist_norm0)
                # кумулятивная метрика экзамплов для одного интента
                    metric += dist_norm0
                    count_sentences += 1
                results_list.append([dist_norm0, clean_norm_example])
    try:
        res = metric * count_sentences/len(list_sentens)
    except:
        res = 0
        
    try:
        res_max = metric_max/count_max
    except:
        res_max = 0
    
    return res + res_max

#------------------------------
#ОСНОВНОЙ ПОИСК ПО НОРМАЛИЗОВАННЫМ EXAMPLAM и RESPONSAM в HIST_BOT_CONFIG_NORM.json
def get_hist_intent(clean_norm_text, text, stop_words):
    results_int = []
    
    for intent in HIST_BOT_CONFIG_NORM['intents'].keys(): 
        list_sentens_examples = HIST_BOT_CONFIG_NORM['intents'][intent]['examples']        
        total_examples_metric = metric_text_in_sentences(clean_norm_text,list_sentens_examples)
        
        list_sentens_responses = HIST_BOT_CONFIG_NORM['intents'][intent]['responses']        
        total_responses_metric = metric_text_in_sentences(clean_norm_text,list_sentens_responses)
        
        #results_int.append([dist_intent+total_examples_metric, intent])
        total_intent_metric = total_examples_metric + total_responses_metric
        results_int.append([total_intent_metric, intent])
    
    # находим минимальное расстояние в непустом массиве избранных интентов    
        
    results_int.sort(key = lambda pair: pair[0],reverse = True)
    print('--'*25)
    print('results_int',results_int[:15])
    print('--'*25)
    #text2voice("Фууфффф... Я очень сильно думала и вот что надумала: ")
    #print("Бутя: Фууфффф... Я очень сильно думала и вот что надумала: ") 

    #выдача
    if (len(results_int)>1) and (results_int[0][0] - results_int[1][0]) < 0.1:
        intent = random.choice(results_int[:2])[1] #неуверенный вывод 17.11
    else:
        intent = results_int[0][1]

    #intent = results_int[0][1]
    #intent = random.choice(results_int[:2])[1] #неуверенный вывод решил использовать 17.11      

    return intent      

## Основа

In [17]:
import wikipedia, re
# Устанавливаем русский язык в Wikipedia
wikipedia.set_lang("ru")
# Чистим текст статьи в Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:1000]
        # Разделяем по точкам
        wikimas=wikitext.split('.')
        # Отбрасываем всЕ после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
        # Если в строке осталось больше трех символов, добавляем ее к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                    wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return random.choice(HIST_BOT_CONFIG['default'])+' (-)'

In [18]:
from transliterate import translit
import time


def bot(text, stop_words):
    text = text.lower()#translit(text, 'ru')
    
    # Если пользователь здоровается
    if text in ('хай','хей','хаюшки','здрасьте','привет', 'здравствуйте', 'доброе утро', 'добрый день', 'добрый вечер', 'доброй ночи','приветствую'):
         return greeting(),"Hellow"       
    
    # Если пользователь не здоровается, тогда мы понимаем, что он делает запрос
    else: 
        
        
        #intent1 = get_chat_intent(text)        
        #print('get_chat_intent:',intent1)
        
        # очищаем запрос от стоп слов
        clear_text = clear_text_stop_words(text, stop_words)
        print('--'*20)
        print('Очищенный запрос: ',clear_text)
        print('--'*20)
        #--------------------------------------
        
        # нормализуем очищенный запрос
        #clean_text = clean0(clear_text)
        clean_norm_text = text_to_normal_form(clear_text)
        
        print('--'*20)
        print('Нормализованный запрос: ',clean_norm_text)
        print('--'*20)
        #--------------------------------------
        
        intent = get_hist_intent(clean_norm_text, text, stop_words)
        

        if (intent != 'unknown_intent') and (intent != 'default1'):
            answer = random.choice(HIST_BOT_CONFIG['intents'][intent]['responses'])
        elif intent == 'default1':
             answer = HIST_BOT_CONFIG['default1'][0]
        else:
            intent1 = get_chat_intent(text)
            if intent1 != 'unknown_intent':
                answer = random.choice(BOT_CONFIG['intents'][intent1]['responses'])
            else:                
                answer = getwiki(clean_norm_text)
                #answer = random.choice(HIST_BOT_CONFIG['default'])+' (-)'
    
    return answer, intent
                

def main_bot(input_text):
    fin1 = open('data/chat_bot/quiest_respons.txt', 'a', encoding='utf-8') 
    #todo как быть, когда одновременно несколько человек общается с ботом
    # вероятно для каждого usera надо создать отдельный лог-файл
    fin1.write(str( datetime.datetime.now() )+'\n')
    #input_text = input('Введите запрос или нажмите Enter для выхода: ')
    
    # открываю и читаю файл со стоп-словами - создаю отсортированный список стоп-слов
    stop_words = open('data/chat_bot/stop-ru.txt', 'r', encoding='utf8')
    stop_words = stop_words.read()
    stop_words = sorted(list(set(stop_words.split('\n'))))    
    
    
    
    #while input_text:   
    start_time = time.time()
    #print('начальное время:', start)
    response = bot(input_text, stop_words)#, fin1 = fin1)    

    print('время ожидания:', time.time() - start_time)
    print('{}'.format(response[0]) )
    print('--'*20)
    #text2voice(response)
    fin1.write(str(datetime.datetime.now())[11:19]+' User: '+input_text+' | '+'Бот: '+response[0]+' | '+response[1]+'\n')
    #input_text = input('Введите запрос или нажмите Enter для выхода: ')
    #print('До свидания')
    #bye_text = random.choice(BOT_CONFIG['intents']['by_seeyou']['responses'])
    #text2voice(bye_text)                     
    #print(bye_text)
    fin1.close()
    return response[0]

### Открываем базу знаний (в т.ч. нормализованная) по предметной области

In [19]:
%%time

import os

import json
# база болтовни
with open ('data/chat_bot/BOT_CONFIG.json', 'r', encoding='utf-8') as f:
    BOT_CONFIG = json.load(f)

# база знаний по истории
with open ('data/chat_bot/HIST_BOT_CONFIG.json', 'r', encoding='utf-8') as f_hist:
    HIST_BOT_CONFIG = json.load(f_hist)  

#база русских стоп-слов    
stop_words = open('data/chat_bot/stop-ru.txt', 'r', encoding='utf8')
stop_words = stop_words.read()
stop_words = sorted(list(set(stop_words.split('\n')))) 

#file_intents_norm('data/skillbox/HIST_BOT_CONFIG_NORM.json')

CPU times: total: 62.5 ms
Wall time: 42 ms


In [20]:
# нормализованная база знаний по истории
with open ('data/chat_bot/HIST_BOT_CONFIG_NORM.json', 'r', encoding='utf-8') as f_hist_norm:
    HIST_BOT_CONFIG_NORM = json.load(f_hist_norm)

### запускаем Геродбота

In [21]:
# код телеграмма
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
#logging.basicConfig(
#    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
#)

#logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    print('update.message.text:', update.message.text)
    try:
        answer = main_bot(update.message.text)
    except:
        answer = "Что-то пошло не так ;("
    
    try:        
        update.message.reply_text(answer)
    except:
        answer = "Что-то сломалось. Извините ;("
        update.message.reply_text(answer)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("2130329456:AAGRRUymCdLRgfm40dOBgOIyNJdwzDjRN10")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

main()

update.message.text: Привет
время ожидания: 0.0
Добрый день
----------------------------------------
update.message.text: Как дела?
----------------------------------------
Очищенный запрос:   
----------------------------------------
----------------------------------------
Нормализованный запрос:  
----------------------------------------
--------------------------------------------------
results_int [[0.0, 'sobiratelstvo_drevnickh_vyzhivanie'], [0.0, 'oxota_drevnickh_vyzhivanie'], [0.0, 'vyzhivanie_drevniy_predki'], [0.0, 'calculus_drevniy_predki'], [0.0, 'obraz_gizni'], [0.0, 'techenia_reki_Kakoe'], [0.0, 'zemledelie_dvukh_Kakoe'], [0.0, 'rodovaia_sosedskaiy_obchina_8786234'], [0.0, 'okhotnikov_bylo_Pochemu'], [0.0, 'Zeronoterka_Zeronoterka_Zeronoterka'], [0.0, 'Priadenie_Priadenie_Priadenie'], [0.0, 'Tkachestvo_Tkachestvo_Tkachestvo'], [0.0, 'Tcinovka_Tcinovka_Tcinovka'], [0.0, 'Dukhi_Dukhi_Dukhi'], [0.0, 'Bogi_Bogi_Bogi']]
--------------------------------------------------
время 

## Тестовый запуск

In [ ]:
привет